In [1]:
from google.colab import files
import pickle

uploaded = files.upload()

Saving tweets-data.csv to tweets-data.csv


In [6]:
import pandas as pd

df = pd.read_csv('tweets-data.csv')
df.head()  # Pour vérifier les colonnes

,Unnamed: 0,Date Created,Number of Likes,Source of Tweet,Tweets,hashtag
0,0,2023-06-25 19:16:20+00:00,0,NaN,@jacksonhinklle #wagner with 6.2 billion dolla...,wagner
1,1,2023-06-25 19:16:18+00:00,0,NaN,Pobrecito es discapacitado\n#Reddetuiterosdemo...,wagner
2,2,2023-06-25 19:16:07+00:00,0,NaN,News from the EIR Daily Alert\n\n“#Putin Addre...,wagner
3,3,2023-06-25 19:15:56+00:00,0,NaN,It's Messi day #Messi𓃵 #Messi36 #Russia #bigst...,wagner
4,4,2023-06-25 19:15:54+00:00,0,NaN,Il passaggio chiave di Machiavelli era questo ...,wagner


In [18]:
import re
import nltk
nltk.download('punkt', force=True)
nltk.download('stopwords', force=True)

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))

def clean_tweet_v2(text):
    text = str(text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#\w+', '', text)
    text = re.sub(r'http\S+|www\S+', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower()
    tokens = text.split()  # Pas de word_tokenize
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [19]:
df['cleaned_text_v2'] = df['Tweets'].apply(clean_tweet_v2)
df[['Tweets', 'cleaned_text_v2']].head()

,Tweets,cleaned_text_v2
0,@jacksonhinklle #wagner with 6.2 billion dolla...,billion dollar
1,Pobrecito es discapacitado\n#Reddetuiterosdemo...,pobrecito es discapacitado
2,News from the EIR Daily Alert\n\n“#Putin Addre...,news eir daily alert addressed people armed ju...
3,It's Messi day #Messi𓃵 #Messi36 #Russia #bigst...,messi day
4,Il passaggio chiave di Machiavelli era questo ...,il passaggio chiave di machiavelli era questo ...


In [21]:
# Installer la bibliothèque Transformers
!pip install transformers --quiet

# Importer le pipeline
from transformers import pipeline

# Charger le pipeline de sentiment (modèle par défaut)
sentiment_pipeline = pipeline("sentiment-analysis")

# Fonction pour appliquer le pipeline et éviter les erreurs de texte trop long
def get_transformer_sentiment(text):
    text = str(text)
    # Tronquer le texte s’il dépasse 1000 caractères (limite BERT ≈ 512 tokens)
    if len(text) > 1000:
        text = text[:1000]
    result = sentiment_pipeline(text)[0]
    return result['label'].lower(), result['score']

# Appliquer la fonction sur la colonne cleaned_text_v2
df[['transformer_sentiment', 'transformer_score']] = df['cleaned_text_v2'].apply(
    lambda x: pd.Series(get_transformer_sentiment(x))
)

# Afficher les premières lignes
df[['cleaned_text_v2', 'transformer_sentiment', 'transformer_score']].head()



No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


,cleaned_text_v2,transformer_sentiment,transformer_score
0,billion dollar,positive,0.999244
1,pobrecito es discapacitado,negative,0.574672
2,news eir daily alert addressed people armed ju...,negative,0.878521
3,messi day,negative,0.999102
4,il passaggio chiave di machiavelli era questo ...,positive,0.850299
